In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

from util.collision_loss import torch_collision_check, NN_constraint_step
from util.zonotope import Zonotope, TorchZonotope
from util.constrained_zonotope import TorchConstrainedZonotope
from util.NN_con_zono import forward_pass_NN_torch, forward_pass_NN_con_zono_torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

# Neural network collision update

In [ ]:
# Toy 2 Layer network
# 2 - 10 - 2

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(2, 10)  
        self.fc2 = nn.Linear(10, 2)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()

In [ ]:
net.to(device)

In [ ]:
# input zonotope
Z_in = TorchZonotope(torch.zeros(2,1).to(device),torch.eye(2).to(device))

# output constraint zonotope ("obstacle")
c_obs = torch.tensor([[0.1],[0.2]]).to(device)
G_obs = torch.diag(torch.tensor([0.05,0.05]).to(device))
Z_obs = TorchConstrainedZonotope(c_obs, G_obs)

# compute initial reachable set
Z_out = forward_pass_NN_torch(Z_in, net)

# plot zonotopes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
for z in Z_out:
    z.plot(ax1)
Z_obs.plot(ax1, 'r')
# plot samples
N_samples = 10000
X_in = np.random.uniform(-1, 1, (N_samples,2))
X_in = torch.as_tensor(X_in, dtype=torch.float).to(device)
Y_out = net(X_in)
ax2.scatter(Y_out[:,0].cpu().detach().numpy(), Y_out[:,1].cpu().detach().numpy())

In [ ]:
# constraint optimizer
con_opt = optim.SGD(net.parameters(), lr=0.1)

torch.autograd.set_detect_anomaly(True)
# # take a constraint step

NN_constraint_step(Z_in, Z_obs, net, con_opt)

# recompute reachable set
Z_out = forward_pass_NN_torch(Z_in, net)

# plot zonotopes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
for z in Z_out:
    z.plot(ax1)
Z_obs.plot(ax1, 'r')
# plot samples
N_samples = 10000
X_in = np.random.uniform(-1, 1, (N_samples,2))
X_in = torch.as_tensor(X_in, dtype=torch.float).to(device)
Y_out = net(X_in)
ax2.scatter(Y_out[:,0].cpu().detach().numpy(), Y_out[:,1].cpu().detach().numpy())

# Neural network function approximator

In [ ]:
# 3 Layer ReLU Feedforward Network
# 2 - 100 - 100 - 2

# n_L1 = 10
# n_L2 = 10

# class Net(nn.Module):

#     def __init__(self):
#         super(Net, self).__init__()
#         # an affine operation: y = Wx + b
#         self.fc1 = nn.Linear(2, n_L2)  
#         self.fc2 = nn.Linear(n_L1, n_L2)
#         self.fc3 = nn.Linear(n_L2, 2)

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# net = Net()

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(2, 10)  
        self.fc2 = nn.Linear(10, 2)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
net.to(device)

In [ ]:
# Use nonlinear function to generate data
# nonlinear function (vectorized)
def f_nonl(x):
    return np.array([x[:,0]**2 + np.sin(x[:,1]),
                     x[:,1]**2 + np.sin(x[:,0])])

# training set
N_train = 10000
X_train = np.random.uniform(-1, 1, (N_train,2))

# compute y's
Y_train = f_nonl(X_train)

X_train = torch.as_tensor(X_train, dtype=torch.float).to(device)
Y_train = torch.as_tensor(Y_train.T, dtype=torch.float).to(device)

In [ ]:
# test set
N_test = 10000
X_test = np.random.uniform(-1, 1, (N_test,2))

# compute y's
Y_test = f_nonl(X_test)

X_test = torch.as_tensor(X_test, dtype=torch.float).to(device)
Y_test = torch.as_tensor(Y_test.T, dtype=torch.float).to(device)

In [ ]:
# train the network
optimizer = optim.SGD(net.parameters(), lr=0.1)

loss = nn.MSELoss()

num_iters = 1000

# training loop:
for i in range(num_iters):
    optimizer.zero_grad()   # zero the gradient buffers

    pred = net(X_train)

    output = loss(pred, Y_train)
    #print('loss: ', output)
    output.backward()
    optimizer.step() 

In [ ]:
Y_test_pred = net(X_test)
test_loss = loss(Y_test_pred, Y_test)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24,8))
color_vec = torch.sum(X_test[:,:2],dim=1).cpu()
ax1.scatter(X_test[:,0].cpu(), X_test[:,1].cpu(),c=color_vec,cmap="gist_rainbow")
ax1.set_title('X test')
ax2.scatter(Y_test[:,0].cpu(), Y_test[:,1].cpu(),c=color_vec,cmap="gist_rainbow")
ax2.set_title('Y test')
ax3.scatter(Y_test_pred[:,0].cpu().detach().numpy(), Y_test_pred[:,1].cpu().detach().numpy(),c=color_vec,cmap="gist_rainbow")
ax3.set_title('Y test pred')

# Constrained training
Train function approximator under some obstacle constraints

In [ ]:
# input zonotope
Z_in = TorchZonotope(torch.zeros(2,1).to(device),torch.eye(2).to(device))

# output constraint zonotope ("obstacle")
c_obs = torch.tensor([[1.5],[1.5]]).to(device)
G_obs = torch.diag(torch.tensor([0.5,0.5]).to(device))
Z_obs = TorchConstrainedZonotope(c_obs, G_obs)

# compute initial reachable set
Z_out = forward_pass_NN_torch(Z_in, net)

# plot zonotopes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
for z in Z_out:
    z.plot(ax1)
Z_obs.plot(ax1, 'r')
# plot samples
N_samples = 10000
X_in = np.random.uniform(-1, 1, (N_samples,2))
X_in = torch.as_tensor(X_in, dtype=torch.float).to(device)
Y_out = net(X_in)
color_vec = torch.sum(X_in,dim=1).cpu()
ax2.scatter(Y_out[:,0].cpu().detach().numpy(), Y_out[:,1].cpu().detach().numpy(),c=color_vec,cmap="gist_rainbow")

In [ ]:
# objective optimizer
obj_opt = optim.SGD(net.parameters(), lr=0.1)

# constraint optimizer
con_opt = optim.SGD(net.parameters(), lr=0.1)

obj = nn.MSELoss()

num_iters = 100

# training loop:
for i in range(num_iters):
    print("iter: ", i)
    # objective update
    obj_opt.zero_grad()   # zero the gradient buffers

    pred = net(X_train)

    obj_loss = obj(pred, Y_train)
    obj_loss.backward()
    obj_opt.step() 

    # constraint update
    NN_constraint_step(Z_in, Z_obs, net, con_opt)

In [ ]:
# recompute reachable set
Z_out = forward_pass_NN_torch(Z_in, net)

# plot zonotopes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
for z in Z_out:
    z.plot(ax1)
Z_obs.plot(ax1, 'r')
# plot samples
N_samples = 10000
X_in = np.random.uniform(-1, 1, (N_samples,2))
X_in = torch.as_tensor(X_in, dtype=torch.float).to(device)
Y_out = net(X_in)
color_vec = torch.sum(X_in,dim=1).cpu()
ax2.scatter(Y_out[:,0].cpu().detach().numpy(), Y_out[:,1].cpu().detach().numpy(),c=color_vec,cmap="gist_rainbow")